# Collect Result

Author: Bingcheng Jin

Date: 2024.1.1

In [1]:
import os
import glob

import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astropy.table import Table, vstack, hstack

In [2]:
band_labels = ['F115W','F150W','F200W','F277W','F356W','F410M','F444W']
nbands = len(band_labels)

In [6]:
sample_base = '../io/sample/'
img_dir_list = glob.glob(sample_base + '*/')
img_dir_list.sort()
img_dir_list

['../../localstore/egs_highz/quad/nircam2/']

In [10]:
with fits.open('../../localstore/egs_highz/quad/nircam2/22246/output.fits') as hdul:
    tab_cheb = Table(hdul['FINAL_CHEB'].data)
    ncheb_re = tab_cheb['COMP1_Re_FIT'].sum()
    cheb_re = list(tab_cheb['COMP1_Re'])[:ncheb_re]
    cheb_re_e = list(tab_cheb['COMP1_Re_ERR'])[:ncheb_re]
    ncheb_n = tab_cheb['COMP1_n_FIT'].sum()
    cheb_n = list(tab_cheb['COMP1_n'])[:ncheb_n]
    cheb_n_e = list(tab_cheb['COMP1_n_ERR'])[:ncheb_n]

    tab_band = Table(hdul['FINAL_BAND'].data)
    n = tab_band['COMP1_n']
cheb_re, cheb_re_e, cheb_n, cheb_n_e, n

([8.0607195, -0.04867809, 0.29724306],
 [0.044726662, 0.10079193, 0.0882077],
 [1.0439999, -0.13136451, 0.9379015],
 [0.00978438, 0.021692524, 0.020381887],
 <Column name='COMP1_n' dtype='float32' length=7>
  2.1132658
  1.3715336
  0.6145034
 0.10778365
  0.4845319
  1.2068254
  1.8505368)

## Before Multi-Component Fit

In [18]:
tab_row = []
for sample_base_dir in img_dir_list:
    pointing = sample_base_dir.split('/')[-2]
    sample_dir_list = glob.glob(sample_base_dir + '*/')
    sample_dir_list.sort()
    for sample_dir in sample_dir_list:
        id = sample_dir.split('/')[-2] # plz consider naming convention
        output_path = sample_dir + 'output_quad.fits'
        if not os.path.exists(output_path):
            print('Output Not Exist in ',sample_dir)
            continue
        with fits.open(output_path) as hdul:
            tab_info = Table(hdul['FIT_INFO'].data)
            fit_sec = tab_info['FITSECT'][0].split(' ')[0]
            chi2nu = float(tab_info['CHI2NU'].data[0])

            tab_band = Table(hdul['FINAL_BAND'].data)
            xc = list(tab_band['COMP1_XC'])
            xc_e = list(tab_band['COMP1_XC_ERR'])
            yc = list(tab_band['COMP1_YC'])
            yc_e = list(tab_band['COMP1_YC_ERR'])
            mag = list(tab_band['COMP1_MAG'])
            mag_e = list(tab_band['COMP1_MAG_ERR'])
            re = list(tab_band['COMP1_Re'])
            re_e = list(tab_band['COMP1_Re_ERR'])
            n = list(tab_band['COMP1_n'])
            n_e = list(tab_band['COMP1_n_ERR'])
            ar = list(tab_band['COMP1_AR'])
            ar_e = list(tab_band['COMP1_AR_ERR'])
            pa = list(tab_band['COMP1_PA'])
            pa_e = list(tab_band['COMP1_PA_ERR'])

            

        row = [id, pointing, chi2nu, fit_sec, xc, xc_e, yc, yc_e, mag, mag_e, re, re_e, n, n_e, ar, ar_e, pa, pa_e]
        tab_row.append(row)
tab_result = Table(rows=tab_row, names=['ID','NIRCamPointing', 'CHI2NU', 'FIT_SECTION', 'XC', 'XC_ERR', 'YC', 'YC_ERR', 'MAG', 'MAG_ERR', 'Re', 'Re_ERR', 'n', 'n_ERR', 'AR', 'AR_ERR', 'PA', 'PA_ERR'])

KeyboardInterrupt: 

### Check Duplication

In [20]:
tab_result = Table.read('../result/egs_quad_multicomp_pls.ecsv')

In [21]:
unq, count = np.unique(tab_result['ID'], return_counts=True)
dup = unq[count > 1]
if len(dup) == 0:
    print('No Duplicated IDs')
else:
    print('Duplicated rows:')
    print(dup)

Duplicated rows:
  ID 
-----
12831
20851
36022


In [ ]:
raise ValueError('Check Duplicated IDs')

## Collect Multi-Component Result

In [12]:
mcomp_dir = '../multicomp/'
tab_row = []
sample_base = '../io/sample/'
img_dir_list = glob.glob(sample_base + 'nircam*/')
for sample_base_dir in img_dir_list:
    pointing = sample_base_dir.split('/')[-2]
    sample_dir_list = glob.glob(sample_base_dir + '*/')
    sample_dir_list.sort()
    for sample_dir in sample_dir_list:
        id = sample_dir.split('/')[-2] # plz consider naming convention
        if os.path.exists(mcomp_dir + id):
            output_path = mcomp_dir + id + '/' + f'output_quad.fits' # important to alter this every time run this cell
            isolate = 0
        else:    # for isolated sources
            output_path = sample_dir + 'output_quad.fits' # important to alter this every time run this cell
            isolate = 1
            
        with fits.open(output_path) as hdul:
            tab_info = Table(hdul['FIT_INFO'].data)
            fit_sec = tab_info['FITSECT'][0].split(' ')[0]
            chi2nu = float(tab_info['CHI2NU'].data[0])

            tab_band = Table(hdul['FINAL_BAND'].data)
            xc = list(tab_band['COMP1_XC'])
            xc_e = list(tab_band['COMP1_XC_ERR'])
            yc = list(tab_band['COMP1_YC'])
            yc_e = list(tab_band['COMP1_YC_ERR'])
            mag = list(tab_band['COMP1_MAG'])
            mag_e = list(tab_band['COMP1_MAG_ERR'])
            re = list(tab_band['COMP1_Re'])
            re_e = list(tab_band['COMP1_Re_ERR'])
            n = list(tab_band['COMP1_n'])
            n_e = list(tab_band['COMP1_n_ERR'])
            ar = list(tab_band['COMP1_AR'])
            ar_e = list(tab_band['COMP1_AR_ERR'])
            pa = list(tab_band['COMP1_PA'])
            pa_e = list(tab_band['COMP1_PA_ERR'])
                
        row = [id, pointing, chi2nu, fit_sec, xc, xc_e, yc, yc_e, mag, mag_e, re, re_e, n, n_e, ar, ar_e, pa, pa_e, isolate]
        tab_row.append(row)
tab_result = Table(rows=tab_row, names=['ID','NIRCamPointing', 'CHI2NU', 'FIT_SECTION', 'XC', 'XC_ERR', 'YC', 'YC_ERR', 'MAG', 'MAG_ERR', 'Re', 'Re_ERR', 'n', 'n_ERR', 'AR', 'AR_ERR', 'PA', 'PA_ERR', 'ISOLATE'])

In [14]:
tab_result.write('../result/egs2_quad_multicomp.ecsv', overwrite=True)

## Tag Point-like Source

In [9]:
plslist = []
#['29463', '25096', '21489', '40413', '40901', '22569', '29745', '14248', '39334', '39904', '36349','35881', '35545', '16488', '38928', '22451', '39385','39128','41123','21722','28756','14748','36445','24830','25457','30345','41153','41217','19448', '19470', '35911','37238', '39314','25566']
print('number of point-like sources:', len(plslist))

number of point-like sources: 0


In [10]:
mcomp_dir = '../multicomp/'
tab_row = []
sample_base = '../io/sample/'
img_dir_list = glob.glob(sample_base + 'nircam*/')
for sample_base_dir in img_dir_list:
    pointing = sample_base_dir.split('/')[-2]
    sample_dir_list = glob.glob(sample_base_dir + '*/')
    sample_dir_list.sort()
    for sample_dir in sample_dir_list:
        id = sample_dir.split('/')[-2] # plz consider naming convention

        pls = 1 if id in plslist else 0
        
        if os.path.exists(mcomp_dir + id):
            output_path = mcomp_dir + id + '/' + f'output_quad.fits' # important to alter this every time run this cell
            isolate = 0
        else:    # for isolated sources
            output_path = sample_dir + 'output_quad.fits' # important to alter this every time run this cell
            isolate = 1
            
        with fits.open(output_path) as hdul:
            tab_info = Table(hdul['FIT_INFO'].data)
            fit_sec = tab_info['FITSECT'][0].split(' ')[0]
            chi2nu = float(tab_info['CHI2NU'].data[0])

            tab_band = Table(hdul['FINAL_BAND'].data)
            xc = list(tab_band['COMP1_XC'])
            xc_e = list(tab_band['COMP1_XC_ERR'])
            yc = list(tab_band['COMP1_YC'])
            yc_e = list(tab_band['COMP1_YC_ERR'])
            mag = list(tab_band['COMP1_MAG'])
            mag_e = list(tab_band['COMP1_MAG_ERR'])
            re = list(tab_band['COMP1_Re'])
            re_e = list(tab_band['COMP1_Re_ERR'])
            n = list(tab_band['COMP1_n'])
            n_e = list(tab_band['COMP1_n_ERR'])
            ar = list(tab_band['COMP1_AR'])
            ar_e = list(tab_band['COMP1_AR_ERR'])
            pa = list(tab_band['COMP1_PA'])
            pa_e = list(tab_band['COMP1_PA_ERR'])

            tab_cheb = Table(hdul['FINAL_CHEB'].data)
            ncheb_re = tab_cheb['COMP1_Re_FIT'].sum()
            cheb_re = list(tab_cheb['COMP1_Re'])[:ncheb_re]
            cheb_re_e = list(tab_cheb['COMP1_Re_ERR'])[:ncheb_re]
            ncheb_n = tab_cheb['COMP1_n_FIT'].sum()
            cheb_n = list(tab_cheb['COMP1_n'])[:ncheb_n]
            cheb_n_e = list(tab_cheb['COMP1_n_ERR'])[:ncheb_n]
                
        row = [id, pointing, chi2nu, fit_sec, xc, xc_e, yc, yc_e, mag, mag_e, re, re_e, n, n_e, ar, ar_e, pa, pa_e, cheb_re, cheb_re_e, cheb_n, cheb_n_e, isolate, pls]
        tab_row.append(row)
tab_result = Table(rows=tab_row, names=['ID','NIRCamPointing', 'CHI2NU', 'FIT_SECTION', 'XC', 'XC_ERR', 'YC', 'YC_ERR', 'MAG', 'MAG_ERR', 'Re', 'Re_ERR', 'n', 'n_ERR', 'AR', 'AR_ERR', 'PA', 'PA_ERR', 'CHEB_Re', 'CHEB_Re_ERR', 'CHEB_n', 'CHEB_n_ERR', 'ISOLATE', 'POINTLIKE'])

ValueError: Arguments "names" and "dtype" must match number of columns

check the table results

In [6]:
tab_result.write(f'../result/egs_quad_multicomp_pls.ecsv', overwrite=True)
tab_result[25:]

ID,NIRCamPointing,CHI2NU,FIT_SECTION,XC,XC_ERR,YC,YC_ERR,MAG,MAG_ERR,Re,Re_ERR,n,n_ERR,AR,AR_ERR,PA,PA_ERR,ISOLATE,POINTLIKE
str5,str7,float64,str13,float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],int64,int64
25258,nircam1,1.054180383682251,"[1:140,1:140]",68.86937 .. 68.335175,0.020454934 .. 0.07228282,69.49339 .. 69.9691,0.021206707 .. 0.07338753,25.612501 .. 26.121618,0.10494478 .. 0.13499908,7.796295 .. 5.7879033,2.0484927 .. 1.8486382,9.921831 .. 7.7983,1.3397316 .. 1.7836874,0.6096759 .. 0.6096759,0.01279053 .. 0.01279053,43.95517 .. 43.95517,1.313281 .. 1.313281,1,0
25324,nircam1,1.1229877471923828,"[1:126,1:126]",63.776558 .. 63.535496,0.021256486 .. 0.020207359,60.19718 .. 63.463512,0.04993814 .. 0.04712853,25.800701 .. 25.16675,0.013736786 .. 0.006451258,6.9750504 .. 7.600315,0.12972131 .. 0.05463658,1.4455689 .. 0.49365366,0.039276198 .. 0.01690287,0.25602305 .. 0.25602305,0.0014625791 .. 0.0014625791,12.167604 .. 12.167604,0.13014732 .. 0.13014732,1,0
25411,nircam1,1.1963496208190918,"[1:174,1:174]",86.403015 .. 86.431,0.012603034 .. 0.008776964,92.04876 .. 92.53128,0.017649949 .. 0.012584961,24.852932 .. 23.747478,0.0061123893 .. 0.0033756834,4.5148873 .. 4.946179,0.029707339 .. 0.017340904,1.1139588 .. 1.1202195,0.013859995 .. 0.010481825,0.6732601 .. 0.6732601,0.0010532828 .. 0.0010532828,-12.978418 .. -12.978418,0.23857005 .. 0.23857005,0,0
25566,nircam1,1.030319094657898,"[1:74,1:74]",37.75564 .. 37.604668,0.020045893 .. 0.025266847,37.73358 .. 37.65197,0.019262714 .. 0.025863122,26.126886 .. 25.970718,0.017213548 .. 0.012465275,1.0123744 .. 1.4547619,0.03344741 .. 0.040050242,3.849068 .. 2.9671068,0.41100782 .. 0.39541408,0.71337986 .. 0.71337986,0.0096691465 .. 0.0096691465,-62.442825 .. -62.442825,1.3416799 .. 1.3416799,1,1
25572,nircam1,1.009782314300537,"[1:92,1:92]",46.77273 .. 46.53223,0.03611555 .. 0.014268294,46.996273 .. 46.86681,0.032624643 .. 0.013372337,26.370455 .. 24.743599,0.027128218 .. 0.0071712765,1.9352033 .. 1.8188456,0.080648996 .. 0.022088036,4.096061 .. 3.3580654,0.3680666 .. 0.17279072,0.74907094 .. 0.74907094,0.006938519 .. 0.006938519,78.34376 .. 78.34376,1.2159702 .. 1.2159702,1,0
26334,nircam1,1.0781608819961548,"[1:172,1:172]",89.97929 .. 87.78804,0.06291346 .. 0.033490386,87.196144 .. 86.84873,0.036784895 .. 0.020216933,26.120415 .. 24.858469,0.023161588 .. 0.0073911906,6.653001 .. 6.736156,0.196614 .. 0.061812572,1.5658613 .. 1.0266454,0.05535063 .. 0.022162814,0.47312862 .. 0.47312862,0.0020986982 .. 0.0020986982,-71.537544 .. -71.537544,0.2682107 .. 0.2682107,1,0
26405,nircam1,1.159672737121582,"[1:138,1:138]",68.69071 .. 68.71838,0.04664702 .. 0.018317124,69.96006 .. 68.250755,0.12445875 .. 0.039518166,25.76355 .. 24.552883,0.011122862 .. 0.0047783493,9.611664 .. 8.240335,0.07867568 .. 0.050010372,0.39242536 .. 0.8951417,0.018277358 .. 0.019179506,0.33593547 .. 0.33593547,0.0010481214 .. 0.0010481214,-9.62175 .. -9.62175,0.11305436 .. 0.11305436,1,0
26553,nircam1,1.09257173538208,"[1:122,1:122]",62.45981 .. 62.015327,0.032333132 .. 0.015210985,61.16048 .. 61.241005,0.028402371 .. 0.01358347,26.337114 .. 24.475925,0.033166256 .. 0.008459231,4.2573915 .. 3.725446,0.25237 .. 0.04952961,3.551054 .. 2.0936406,0.22751103 .. 0.07008264,0.8094581 .. 0.8094581,0.005105861 .. 0.005105861,-54.07792 .. -54.07792,1.0417063 .. 1.0417063,1,0
27129,nircam1,1.1271311044692993,"[1:106,1:106]",54.113518 .. 53.576374,0.021473208 .. 0.024217656,53.33325 .. 52.680126,0.036860533 .. 0.03566227,26.121292 .. 25.34885,0.008235161 .. 0.006397053,4.561341 .. 4.6834445,0.03254333 .. 0.037479624,0.24259552 .. 0.5439451,0.01686479 .. 0.028108077,0.31949624 .. 0.31949624,0.0024521542 .. 0.0024521542,-23.36228 .. -23.36228,0.18221134 .. 0.18221134,1,0


In [15]:
tab_result[tab_result['ID'] == '31079'][0]['n']

array([1.930769 , 2.1653936, 2.4315352, 2.6979735, 2.7732017, 2.7130527,
       2.6325293], dtype=float32)

## Patch

In [11]:
tab_p = Table.read('../result/egs_quad_multicomp_pls.ecsv')

In [12]:
tab_row = []
for row in tab_p:
    id = row['ID']
    pointing = row['NIRCamPointing']
    if row['ISOLATE'] == 1:
        if pointing in ['nircam1', 'nircam2', 'nircam3', 'nircam6']:
            output_path = f'../../localstore/egs_highz/quad/{pointing}/{id}/output.fits'
        else:
            output_path = f'../../localstore/egs_highz/quad/{pointing}/{id}/output_quad.fits'
    else:
        output_path = f'../../localstore/egs_highz/quad/multicomp/{id}/output_quad.fits'
    
    with fits.open(output_path) as hdul:
        tab_cheb = Table(hdul['FINAL_CHEB'].data)
        ncheb_re = tab_cheb['COMP1_Re_FIT'].sum()
        cheb_re = list(tab_cheb['COMP1_Re'])[:ncheb_re]
        cheb_re_e = list(tab_cheb['COMP1_Re_ERR'])[:ncheb_re]
        ncheb_n = tab_cheb['COMP1_n_FIT'].sum()
        cheb_n = list(tab_cheb['COMP1_n'])[:ncheb_n]
        cheb_n_e = list(tab_cheb['COMP1_n_ERR'])[:ncheb_n]
    
    row = [cheb_re, cheb_re_e, cheb_n, cheb_n_e]
    tab_row.append(row)
tab_cheb = Table(rows=tab_row, names=['CHEB_Re', 'CHEB_Re_ERR', 'CHEB_n', 'CHEB_n_ERR'])
tab_patched = hstack([tab_p, tab_cheb])

In [13]:
tab_patched.write('../result/egs_quad_multicomp_pls_cheb.ecsv', overwrite=True)